# **Sentiment Analysis on Reddit**

In [1]:
import praw
import pandas as pd
import nltk
import fuzzywuzzy as fuzz

import bot1

reddit = praw.Reddit(client_id=bot1.app_id, client_secret=bot1.app_secret, user_agent=bot1.app_ua)
sub = 'pcgaming'
subreddit = reddit.subreddit(sub)
topic_list = []
num_posts = 5

def get_topic(topic_list):
    topic = ''

    return topic

def score_comments(url):
    pass

for post in subreddit.hot(limit=num_posts):
    url = 'reddit.com/r/' + sub + '/comments/' + post.id
    print(url)
    score_comments(url)



reddit.com/r/pcgaming/comments/vkw2bb
reddit.com/r/pcgaming/comments/vmdw2l
reddit.com/r/pcgaming/comments/vmky7u
reddit.com/r/pcgaming/comments/vmmwrp
reddit.com/r/pcgaming/comments/vmms5j
